In [ ]:
# !pip3 install auto-sklearn

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score 
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autosklearn.classification import AutoSklearnClassifier
from google.colab import drive
import pathlib
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autosklearn.classification import AutoSklearnClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.over_sampling import SMOTE
import pickle

In [ ]:
# NOTE: doing feature_wiz in the same notebook of autoML is giving me a lot of conflicts, 
# so i'm moving the code that does feature_wiz to another notebook where you can check that the 
# features it choose are:
features=['delivery_performance',
 'number_of_itens',
 'payment_type_boleto',
 'order_status_shipped',
 'order_status_processing',
 'avg_dist_by_order_w_by_price',
 'total_freight_value',
 'purchase_delivery_days',
 'mean_freight_value',
 'median_price',
 'order_status_delivered',
 'mean_product_description_lenght',
 'flag_outros',
 'flag_moda',
 'total_weight',
 'flag_moveis',
 'flag_eltroeletronicos',
 'flag_ferramentas',
 'total_cost',
 'flag_consumiveis',
 'payment_type_voucher',
 'order_status_canceled',
 'order_status_invoiced']

In [ ]:
drive.mount('/content/drive')
# Data sources drive folder 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Normalized

In [ ]:
# parquet_path = pathlib.Path("/content/drive/MyDrive/mestrado/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/final_dataset.parquet")
parquet_path = pathlib.Path("/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/final_dataset_norm.parquet")

## All Features

In [ ]:
dataset=pd.read_parquet(parquet_path)
dataset=pd.get_dummies(dataset, columns=['order_status'])
dataset=dataset.drop(columns=['order_id','customer_id','customer_unique_id','review_id',
                              'order_purchase_timestamp','order_approved_at',
                              'order_delivered_carrier_date','order_delivered_customer_date',
                              'order_estimated_delivery_date','customer_city',
                              'customer_state','customer_zip_code_prefix',
                              'review_comment_title', 'review_comment_message',
                              'review_creation_date', 'review_answer_timestamp',
                              'geolocation_zip_code_prefix', 'cust_lat', 'cust_lng', 
                              'cust_city','cust_state','payment_sequential'])
dataset = dataset.dropna()
dataset['review_binary']=dataset.apply(lambda x: 1 if x['review_score']>3 else 0,axis=1)
dataset=dataset.drop(columns='review_score')
dataset.columns

Index(['delivery_performance', 'purchase_delivery_days',
       'payment_installments', 'payment_value', 'avg_distance',
       'avg_dist_by_order_w_by_price', 'number_of_itens', 'mean_price',
       'median_price', 'sum_price', 'total_freight_value',
       'mean_freight_value', 'total_cost', 'total_weight',
       'mean_product_description_lenght', 'mean_product_name_lenght',
       'mean_product_photos_qty', 'payment_type_boleto',
       'payment_type_credit_card', 'payment_type_debit_card',
       'payment_type_not_defined', 'payment_type_voucher', 'flag_consumiveis',
       'flag_moveis', 'flag_eltroeletronicos', 'flag_moda', 'flag_ferramentas',
       'flag_outros', 'order_status_approved', 'order_status_canceled',
       'order_status_delivered', 'order_status_invoiced',
       'order_status_processing', 'order_status_shipped',
       'order_status_unavailable', 'review_binary'],
      dtype='object')

In [ ]:
y=dataset['review_binary']
X=dataset.drop(columns='review_binary')

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define search
model = AutoSklearnClassifier(time_left_for_this_task=60*60*2, per_run_time_limit=60*10, n_jobs=8,
                              disable_evaluator_output=False, resampling_strategy='cv', 
                              resampling_strategy_arguments={'folds': 5})
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)
f1=f1_score(y_test, y_hat)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
specificity = tn / (tn+fp)
sensitivity = (tp)/(tp+ fn)
print("Accuracy: %.3f" % acc)
print("F1-score: %.3f" % f1)
print("specificity: %.3f" % specificity)
print("sensitivity: %.3f" % sensitivity)

auto-sklearn results:
  Dataset name: a7cc63be-0897-11ed-8214-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.876877
  Number of target algorithm runs: 120
  Number of successful target algorithm runs: 49
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 64
  Number of target algorithms that exceeded the memory limit: 7

Accuracy: 0.882
Accuracy: 0.882
F1-score: 0.891
specificity: 0.802
sensitivity: 0.962


In [ ]:
with open("/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/models/norm_smote_all_features_extended.pkl", 'wb') as files:
    pickle.dump(model, files)

## Feature Selection

In [ ]:
y=dataset['review_binary']
X=dataset[[*features]]

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# define search
model = AutoSklearnClassifier(time_left_for_this_task=60*60*2, per_run_time_limit=60*10, n_jobs=8,
                              disable_evaluator_output=False, resampling_strategy='cv', 
                              resampling_strategy_arguments={'folds': 5})
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)
f1=f1_score(y_test, y_hat)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
specificity = tn / (tn+fp)
sensitivity = (tp)/(tp+ fn)
print("Accuracy: %.3f" % acc)
print("F1-score: %.3f" % f1)
print("specificity: %.3f" % specificity)
print("sensitivity: %.3f" % sensitivity)

auto-sklearn results:
  Dataset name: 7e3a2e66-08a8-11ed-8214-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.862008
  Number of target algorithm runs: 123
  Number of successful target algorithm runs: 58
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 59
  Number of target algorithms that exceeded the memory limit: 4

Accuracy: 0.872
Accuracy: 0.872
F1-score: 0.880
specificity: 0.801
sensitivity: 0.943


In [ ]:
with open("/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/models/norm_smote_selected_features_extended.pkl", 'wb') as files:
    pickle.dump(model, files)

# Not Normalized

In [ ]:
# NOTE: doing feature_wiz in the same notebook of autoML is giving me a lot of conflicts, 
# so i'm moving the code that does feature_wiz to another notebook where you can check that the 
# features it choose are:
features=['delivery_performance',
 'number_of_itens',
 'payment_type_boleto',
 'order_status_shipped',
 'order_status_processing',
 'avg_dist_by_order_w_by_price',
 'total_freight_value',
 'purchase_delivery_days',
 'mean_freight_value',
 'median_price',
 'order_status_delivered',
 'flag_eltroeletronicos',
 'mean_product_description_lenght',
 'mean_product_photos_qty',
 'flag_moveis',
 'flag_outros',
 'flag_moda',
 'flag_ferramentas',
 'mean_product_name_lenght',
 'order_status_canceled',
 'order_status_invoiced']

In [ ]:
# Data sources drive folder 
parquet_path = pathlib.Path("/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/final_dataset.parquet")

## All Features

In [ ]:
dataset=pd.read_parquet(parquet_path)
dataset=pd.get_dummies(dataset, columns=['order_status'])
dataset=dataset.drop(columns=['order_id','customer_id','customer_unique_id','review_id',
                              'order_purchase_timestamp','order_approved_at',
                              'order_delivered_carrier_date','order_delivered_customer_date',
                              'order_estimated_delivery_date','customer_city',
                              'customer_state','customer_zip_code_prefix',
                              'review_comment_title', 'review_comment_message',
                              'review_creation_date', 'review_answer_timestamp',
                              'geolocation_zip_code_prefix', 'cust_lat', 'cust_lng', 
                              'cust_city','cust_state','payment_sequential'])
dataset=dataset.dropna()
dataset['review_binary']=dataset.apply(lambda x: 1 if x['review_score']>3 else 0,axis=1)
dataset=dataset.drop(columns='review_score')

dataset.columns

Index(['delivery_performance', 'purchase_delivery_days',
       'payment_installments', 'payment_value', 'payment_type_boleto',
       'payment_type_credit_card', 'payment_type_debit_card',
       'payment_type_not_defined', 'payment_type_voucher', 'avg_distance',
       'avg_dist_by_order_w_by_price', 'number_of_itens', 'mean_price',
       'median_price', 'sum_price', 'total_freight_value',
       'mean_freight_value', 'total_cost', 'total_weight',
       'mean_product_description_lenght', 'mean_product_name_lenght',
       'mean_product_photos_qty', 'flag_consumiveis', 'flag_moveis',
       'flag_eltroeletronicos', 'flag_moda', 'flag_ferramentas', 'flag_outros',
       'order_status_approved', 'order_status_canceled',
       'order_status_delivered', 'order_status_invoiced',
       'order_status_processing', 'order_status_shipped',
       'order_status_unavailable', 'review_binary'],
      dtype='object')

In [ ]:
y=dataset['review_binary']
X=dataset.drop(columns='review_binary')
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:

# define search
model = AutoSklearnClassifier(time_left_for_this_task=60*60*2, per_run_time_limit=60*10, n_jobs=8,
                              disable_evaluator_output=False, resampling_strategy='cv', 
                              resampling_strategy_arguments={'folds': 5})
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)
f1=f1_score(y_test, y_hat)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
specificity = tn / (tn+fp)
sensitivity = (tp)/(tp+ fn)
print("Accuracy: %.3f" % acc)
print("F1-score: %.3f" % f1)
print("specificity: %.3f" % specificity)
print("sensitivity: %.3f" % sensitivity)

auto-sklearn results:
  Dataset name: 6038ac8a-08b9-11ed-8214-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.837450
  Number of target algorithm runs: 143
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 58
  Number of target algorithms that exceeded the memory limit: 7

Accuracy: 0.872
Accuracy: 0.872
F1-score: 0.882
specificity: 0.787
sensitivity: 0.957


In [ ]:
with open('/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/models/no_norm_smote_all_feature_model_extended.pkl', 'wb') as files:
    pickle.dump(model, files)

## Feature Selection

In [ ]:
y=dataset['review_binary']
X=dataset[[*features]]
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
model = AutoSklearnClassifier(time_left_for_this_task=60*60*2, per_run_time_limit=60*10, n_jobs=8,
                              disable_evaluator_output=False, resampling_strategy='cv', 
                              resampling_strategy_arguments={'folds': 5})
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)
f1=f1_score(y_test, y_hat)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
specificity = tn / (tn+fp)
sensitivity = (tp)/(tp+ fn)
print("Accuracy: %.3f" % acc)
print("F1-score: %.3f" % f1)
print("specificity: %.3f" % specificity)
print("sensitivity: %.3f" % sensitivity)

auto-sklearn results:
  Dataset name: 31ab6b6c-08ca-11ed-8214-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.849929
  Number of target algorithm runs: 135
  Number of successful target algorithm runs: 62
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 65
  Number of target algorithms that exceeded the memory limit: 7

Accuracy: 0.853
Accuracy: 0.853
F1-score: 0.864
specificity: 0.770
sensitivity: 0.936


In [ ]:
with open('/content/drive/MyDrive/ICPR 2022 - Brazilian E-Commerce Public Dataset by Olist/models/no_norm_smote_feature_selected_model_extended.pkl', 'wb') as files:
    pickle.dump(model, files)